## 한글 단어 리트리버 튜닝

In [46]:
from langchain_openai import OpenAIEmbeddings

In [ ]:
# !pip install kiwipiepy

In [47]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [48]:
kiwi.tokenize("안녕하세요? 형태소 분석기 키위입니다")

[Token(form='안녕', tag='NNG', start=0, len=2),
 Token(form='하', tag='XSA', start=2, len=1),
 Token(form='세요', tag='EF', start=3, len=2),
 Token(form='?', tag='SF', start=5, len=1),
 Token(form='형태소', tag='NNG', start=7, len=3),
 Token(form='분석기', tag='NNG', start=11, len=3),
 Token(form='키위', tag='NNG', start=15, len=2),
 Token(form='이', tag='VCP', start=17, len=1),
 Token(form='ᆸ니다', tag='EF', start=17, len=3)]

In [49]:
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from langchain.vectorstores import FAISS


docs = [
    Document(
        page_content="금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다."
    ),
    Document(
        page_content="금융저축보험은 규칙적인 저축을 통해 목돈을 마련할 수 있으며, 생명보험 기능도 겸비하고 있습니다."
    ),
    Document(
        page_content="저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다."
    ),
    Document(
        page_content="금융저축산물보험은 장기적인 저축 목적과 더불어, 축산물 제공 기능을 갖추고 있는 특별 금융 상품입니다."
    ),
    Document(
        page_content="금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다."
    ),
    Document(
        page_content="금보험은 저축성과를 극대화합니다. 특히 노후 대비 저축에 유리하게 구성되어 있습니다."
    ),
    Document(
        page_content="금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요."
    ),
]
# retriever = BM25Retriever.from_documents()

In [50]:
for doc in docs:
    print(" ".join([token.form for token in kiwi.tokenize(doc.page_content)]))

금융 보험 은 장기 적 이 ᆫ 자산 관리 와 위험 대비 를 목적 으로 고안 되 ᆫ 금융 상품 이 ᆸ니다 .
금융 저축 보험 은 규칙 적 이 ᆫ 저축 을 통하 어 목돈 을 마련 하 ᆯ 수 있 으며 , 생명 보험 기능 도 겸비 하 고 있 습니다 .
저축 금융 보험 은 저축 과 금융 을 통하 어 목돈 마련 에 도움 을 주 는 보험 이 ᆸ니다 . 또한 , 사망 보장 기능 도 제공 하 ᆸ니다 .
금융 저 축산물 보험 은 장기 적 이 ᆫ 저축 목적 과 더불 어 , 축산물 제공 기능 을 갖추 고 있 는 특별 금융 상품 이 ᆸ니다 .
금융 단 폭격 보험 은 저축 은 커녕 위험 대비 에 초점 을 맞추 ᆫ 상품 이 ᆸ니다 . 높 은 위험 을 감수 하 고자 하 는 고객 에게 적합 하 ᆸ니다 .
금 보험 은 저축 성과 를 극대 화 하 ᆸ니다 . 특히 노후 대비 저축 에 유리 하 게 구성 되 어 있 습니다 .
금융 보 씨 험하 ᆫ 말 좀 하 지 말 시 고 , 저축 이나 좀 하 시 던가요 . 뭐 가 그리 급하 시 ᆫ지 모르 겠 네요 .


In [ ]:
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [51]:
bm25 = BM25Retriever.from_documents(docs)
kiwi_bm25 = BM25Retriever.from_documents(docs, preprocess_func=kiwi_tokenize)
faiss = FAISS.from_documents(docs, OpenAIEmbeddings()).as_retriever()
bm25_faiss_73 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
bm25_faiss_37 = EnsembleRetriever(
    retrievers=[bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_73 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
}

In [52]:
def print_search_results(retrievers, query):
    print(f"Query: {query}")
    for name, retriever in retrievers.items():
        print(f"{name}    \t: {retriever.invoke(query)[0].page_content}")
    print("===" * 20)

In [53]:
print_search_results(retrievers, "금융보험")
print_search_results(retrievers, "금융 보험")
print_search_results(retrievers, "금융저축보험")
print_search_results(retrievers, "축산물 보험")
print_search_results(retrievers, "저축금융보험")
print_search_results(retrievers, "금융보씨 개인정보 조회")

Query: 금융보험
bm25    	: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
kiwi_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
faiss    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_73    	: 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
bm25_faiss_37    	: 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
kiwi_bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Query: 금융 보험
bm25    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
faiss    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Query: 금융저축보험
bm25 

## Konlpy

In [ ]:
# !pip install konlpy

In [54]:
from konlpy.tag import Kkma, Okt

kkma = Kkma()
okt = Okt()

In [55]:
kkma.morphs("안녕하세요? 형태소 분석기 꼬꼬마입니다")

['안녕', '하', '세요', '?', '형태소', '분석기', '꼬꼬마', '이', 'ㅂ니다']

In [56]:
kkma.nouns("안녕하세요? 형태소 분석기 꼬꼬마입니다")

['안녕', '형태소', '분석기', '꼬꼬마']

In [57]:
okt.morphs("안녕하세요? 형태소 분석기 오케이티입니다")

['안녕하세요', '?', '형태소', '분석', '기', '오', '케이티', '입니다']

In [58]:
def kkma_tokenize(text):
    return [token for token in kkma.morphs(text)]

In [59]:
def okt_tokenize(text):
    return [token for token in okt.morphs(text)]

In [60]:
kkma_bm25 = BM25Retriever.from_documents(docs, preprocess_func=kkma_tokenize)
kkma_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
kkma_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[kkma_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

okt_bm25 = BM25Retriever.from_documents(docs, preprocess_func=okt_tokenize)
okt_bm25_faiss_73 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.7, 0.3],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)
okt_bm25_faiss_37 = EnsembleRetriever(
    retrievers=[okt_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

retrievers = {
    "bm25": bm25,
    "kiwi_bm25": kiwi_bm25,
    "faiss": faiss,
    "bm25_faiss_73": bm25_faiss_73,
    "bm25_faiss_37": bm25_faiss_37,
    "kiwi_bm25_faiss_73": kiwibm25_faiss_73,
    "kiwi_bm25_faiss_37": kiwibm25_faiss_37,
    "kkma_bm25": kkma_bm25,
    "kkma_bm25_faiss_73": kkma_bm25_faiss_73,
    "kkma_bm25_faiss_37": kkma_bm25_faiss_37,
    "okt_bm25": okt_bm25,
    "okt_bm25_faiss_73": okt_bm25_faiss_73,
    "okt_bm25_faiss_37": okt_bm25_faiss_37,
}

In [61]:
print_search_results(retrievers, "금융보험")
print_search_results(retrievers, "금융 보험")
print_search_results(retrievers, "금융저축보험")
print_search_results(retrievers, "축산물 보험")
print_search_results(retrievers, "저축금융보험")
print_search_results(retrievers, "금융보씨 개인정보 조회")

Query: 금융보험
bm25    	: 금융보씨 험한말 좀 하지마시고, 저축이나 좀 하시던가요. 뭐가 그리 급하신지 모르겠네요.
kiwi_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
faiss    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
bm25_faiss_73    	: 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
bm25_faiss_37    	: 금융단폭격보험은 저축은 커녕 위험 대비에 초점을 맞춘 상품입니다. 높은 위험을 감수하고자 하는 고객에게 적합합니다.
kiwi_bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kiwi_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kkma_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
kkma_bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
kkma_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
okt_bm25    	: 저축금융보험은 저축과 금융을 통해 목돈 마련에 도움을 주는 보험입니다. 또한, 사망 보장 기능도 제공합니다.
okt_bm25_faiss_73    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
okt_bm25_faiss_37    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금융 상품입니다.
Query: 금융 보험
bm25    	: 금융보험은 장기적인 자산 관리와 위험 대비를 목적으로 고안된 금